### Set Path

In [34]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

### Import Packages and Functions

In [ ]:
from src.option_class import Option
from src.models.black_scholes import BlackScholes
from src.models.monte_carlo import MonteCarlo
from src.models.binomial import Binomial
from src.fetch_data import StockOptionsData
from src.arbitrage.detector import ArbitrageDetector
from analysis.iv_solver import ImpliedVolatilitySolver
from analysis.fd_greeks import FiniteDifferenceGreeks

import pandas as pd
import json as js

### Example Option Data Calculation

In [36]:
option_type = "call"
S = 100
K = 100
T = 1
sigma = 0.2
r = 0.05
q = 0

option = Option(
    option_type=option_type,
    S=S,
    K=K,
    T=T,
    sigma=sigma,
    r=r,
    q=q
)

### Black-Scholes Price

In [37]:
model_bs = BlackScholes(option)
price_bs = model_bs.price()
print(f"Black Scholes price: ${price_bs:.4f}")

Black Scholes price: $10.4506


### Monte Carlo Price

In [38]:
n_steps = 1_000
n_paths = 100_000

model_mc = MonteCarlo(option, n_steps, n_paths)
price_mc, std_error_mc = model_mc.price()
print(f"Monte Carlo price: ${price_mc:.4f} with standard error of {std_error_mc:.4f}")

Monte Carlo price: $10.4398 with standard error of 0.0465


### Binomial Price

In [39]:
option_style = "american"
N = 1_000

model_bin = Binomial(option, option_style, N)

price_crr = model_bin.price("crr")
price_jr = model_bin.price("jr")
price_lr = model_bin.price("lr")

print(f"Cox-Ross-Rubinstein price: ${price_crr:.4f}")
print(f"Jarrow-Rudd price: ${price_jr:.4f}")
print(f"Leisen-Reimer price: ${price_lr:.4f}")

Cox-Ross-Rubinstein price: $10.4486
Jarrow-Rudd price: $10.4522
Leisen-Reimer price: $10.4469


### Implied Volatilities

In [40]:
prices = [price_bs, price_mc, price_crr, price_jr, price_lr]

rows = ["Black-Scholes", "Monte-Carlo", "Cox-Ross-Rubinstein", "Jarrow-Rudd", "Leisen-Reimer"]
cols = ["Newton Raphson", "Brent"]

iv_df = pd.DataFrame(index=rows, columns=cols)

for row_name, price in zip(rows, prices):
    iv_solver = ImpliedVolatilitySolver(price, option_type, S, K, T, r, q)

    iv_newton_raphson = iv_solver.iv_newton_raphson()
    iv_brent = iv_solver.iv_brent()

    iv_df.loc[row_name] = [iv_newton_raphson, iv_brent]

display(iv_df)

,Newton Raphson,Brent
Black-Scholes,0.2,0.2
Monte-Carlo,0.199713,0.199713
Cox-Ross-Rubinstein,0.199947,0.199947
Jarrow-Rudd,0.200043,0.200043
Leisen-Reimer,0.199901,0.199901


### Greeks

In [41]:
rows = ["Black-Scholes", "Cox-Ross-Rubinstein", "Jarrow-Rudd", "Leisen-Reimer"]
cols = ["Delta", "Gamma", "Vega", "Theta", "Rho"]
greeks_df = pd.DataFrame(index=rows, columns=cols)

greeks_bs = model_bs.greeks()
greeks_df.loc["Black-Scholes"] = greeks_bs

fd = FiniteDifferenceGreeks(option, option_style, N)
pricing_models = [Binomial._crr_model, Binomial._jr_model, Binomial._lr_model]

for row_name, model in zip(rows[1:], pricing_models):
    fd_delta = fd.fd_delta(model, 0.01*S)
    fd_gamma = fd.fd_gamma(model, 0.02*S)
    fd_vega = fd.fd_vega(model, 0.01) / 100
    fd_theta = -fd.fd_theta(model, 1/365) / 365
    fd_rho = fd.fd_rho(model, 1e-4) / 100

    greeks_df.loc[row_name] = [fd_delta, fd_gamma, fd_vega, fd_theta, fd_rho]

display(greeks_df)

,Delta,Gamma,Vega,Theta,Rho
Black-Scholes,0.636831,0.018762,0.37524,-0.017573,0.532325
Cox-Ross-Rubinstein,0.636799,0.02057,0.375116,-0.01757,0.532313
Jarrow-Rudd,0.635898,0.017885,0.374243,-0.017587,0.535363
Leisen-Reimer,0.636778,0.018761,0.375012,-0.017568,0.532396


### Put-Call Parity Arbitrage

In [42]:
S = 100
K = 100
T = 1
sigma = 0.2
r = 0.05
q = 0

call = Option("call", S, K, T, sigma, r, q)
put = Option("put", S, K, T, sigma, r, q)

call_bs = BlackScholes(call)
put_bs = BlackScholes(put)

C = call_bs.price() * 1.2
P = put_bs.price()

ad = ArbitrageDetector()

pcp = ad.put_call_parity(C, P, S, K, T, r)
print(js.dumps(pcp, indent=4))

{
    "call price": 12.540700286622677,
    "put price": 5.573526022256971,
    "arbitrage exists": true,
    "profit": 2.0901167144371158,
    "strategy": "Conversion (call is overpriced)",
    "details": {
        "action": [
            "Buy 1 put option",
            "Buy 1 share of stock",
            "Sell 1 call option",
            "Lend $95.12 at risk-free rate 5.00%"
        ],
        "Initial Inflow": 2.0901167144371158
    }
}


### Fetch Real Data

In [43]:
ticker = 'aapl'

stock = StockOptionsData(ticker)
expirations = stock.get_expirations()
display(expirations)

call_options = stock.get_call_options()
display(call_options[:5])

put_options = stock.get_put_options()
display(put_options[:5])

option_data, option_chain = stock.get_complete_options_data()
option_data = pd.DataFrame(option_data)
display(option_data)

option_chain = option_chain.fillna('-')
display(option_chain[:5])

option, market_price = stock.get_relevant_options_data("call")

model_bs = BlackScholes(option)
price_bs = model_bs.price()

model_mc = MonteCarlo(option)
price_mc, _ = model_mc.price()

rows = ["Market", "Black-Scholes", "Monte Carlo", "Cox-Ross-Rubinstein", "Jarrow-Rudd", "Leisen-Reimer"]
cols = [f"{option.option_type} option @ {option.K}"]
prices_df = pd.DataFrame(index=rows, columns=cols)

prices_df.loc["Market"] = market_price
prices_df.loc["Black-Scholes"] = price_bs
prices_df.loc["Monte Carlo"] = price_mc

bin_pricing_models = ["CRR", "JR", "LR"]

for row_name, model in zip(rows[3:], bin_pricing_models):
    model_bin = Binomial(option, "american")
    price = model_bin.price(model)

    prices_df.loc[row_name] = price

display(prices_df)

display(pd.DataFrame(option.data()))

solver = ImpliedVolatilitySolver(market_price, option.option_type, option.S, option.K, option.T, option.r, option.q)
vol_nr = solver.iv_newton_raphson()
vol_brent = solver.iv_brent()
print(f"Implied Volatility (Newton-Raphson): {vol_nr}")
print(f"Implied Volatility (Brent): {vol_brent}")

['2026-02-23',
 '2026-02-25',
 '2026-02-27',
 '2026-03-06',
 '2026-03-13',
 '2026-03-20',
 '2026-03-27',
 '2026-04-02',
 '2026-04-17',
 '2026-05-15',
 '2026-06-18',
 '2026-07-17',
 '2026-08-21',
 '2026-09-18',
 '2026-11-20',
 '2026-12-18',
 '2027-01-15',
 '2027-06-17',
 '2027-12-17',
 '2028-01-21',
 '2028-03-17',
 '2028-12-15']

,contractSymbol,lastTradeDate,strike,price,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL260223C00220000,2026-02-20 20:09:47+00:00,220.0,44.675,43.70,45.65,2.119999,5.002357,5,5,1.126957,True,REGULAR,USD
1,AAPL260223C00225000,2026-02-20 19:56:20+00:00,225.0,39.600,38.55,40.65,4.280003,12.330749,1,8,0.828127,True,REGULAR,USD
2,AAPL260223C00230000,2026-02-18 16:34:47+00:00,230.0,34.625,33.60,35.65,0.000000,0.000000,1,7,0.800783,True,REGULAR,USD
3,AAPL260223C00235000,2026-02-20 19:51:35+00:00,235.0,29.600,28.55,30.65,-0.959999,-3.201064,4,10,0.625004,True,REGULAR,USD
4,AAPL260223C00240000,2026-02-20 20:47:18+00:00,240.0,24.625,23.85,25.40,1.900001,8.715604,23,10,0.582035,True,REGULAR,USD


,contractSymbol,lastTradeDate,strike,price,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL260223P00210000,2026-02-20 20:26:58+00:00,210.0,0.005,0.0,0.01,-0.02,-66.666670,792.0,10.0,1.000005,False,REGULAR,USD
1,AAPL260223P00215000,2026-02-20 20:26:50+00:00,215.0,0.005,0.0,0.01,0.00,0.000000,692.0,231.0,0.906251,False,REGULAR,USD
2,AAPL260223P00220000,2026-02-20 20:31:58+00:00,220.0,0.005,0.0,0.01,-0.01,-50.000000,4.0,208.0,0.812502,False,REGULAR,USD
3,AAPL260223P00225000,2026-02-19 20:00:21+00:00,225.0,0.040,0.0,0.08,0.00,0.000000,7.0,75.0,0.898439,False,REGULAR,USD
4,AAPL260223P00230000,2026-02-20 14:33:02+00:00,230.0,0.040,0.0,0.08,-0.01,-33.333336,11.0,414.0,0.789065,False,REGULAR,USD


,Value
Ticker,AAPL
Stock Price,264.579987
Expiration Date,2026-02-23
Time to Maturity,0.00274
Risk Free Rate,0.04086
Dividend Yield,0.0039


,callPrice,callBid,callAsk,callVolume,callOpenInterest,callImpliedVolatility,strike,putPrice,putBid,putAsk,putVolume,putOpenInterest,putImpliedVolatility
0,-,-,-,-,-,-,210.0,0.005,0.0,0.01,792.0,10.0,1.000005
1,-,-,-,-,-,-,215.0,0.005,0.0,0.01,692.0,231.0,0.906251
2,44.675,43.7,45.65,5.0,5.0,1.126957,220.0,0.005,0.0,0.01,4.0,208.0,0.812502
3,39.6,38.55,40.65,1.0,8.0,0.828127,225.0,0.04,0.0,0.08,7.0,75.0,0.898439
4,34.625,33.6,35.65,1.0,7.0,0.800783,230.0,0.04,0.0,0.08,11.0,414.0,0.789065


,call option @ 220.0
Market,44.675
Black-Scholes,44.605198
Monte Carlo,44.603437
Cox-Ross-Rubinstein,44.605174
Jarrow-Rudd,44.605156
Leisen-Reimer,44.605179


,Value
Option Type,call
Stock Price,264.579987
Strike Price,220.0
Time to Maturity,0.00274
Volatility,1.126957
Risk-free Rate,0.04086
Dividend Yield,0.0039


Newton-Rapshon implied volatility did not converge
Implied Volatility (Newton-Raphson): None
Implied Volatility (Brent): 1.5400425381529885


### Calculate Greeks

In [44]:
greeks = model_bs.greeks()
display(greeks)

{'Delta': np.float64(0.9991984614857944),
 'Gamma': np.float64(0.00017388519134979694),
 'Vega': np.float64(0.00037582983014717463),
 'Theta': np.float64(-0.042953841386818306),
 'Rho': np.float64(0.006020896367255738)}

### Compare Market Price With Black-Scholes Price

In [45]:
ad = ArbitrageDetector(0.2)
bs_arb = ad.market_price_vs_bs_price(market_price, option)

print(js.dumps(bs_arb, indent=4))

{
    "market price": 44.675,
    "black-scholes price": 44.605198118105506,
    "delta": 0.9991984614857944,
    "arbitrage exists": false,
    "profit": -61.738781222693504
}


### Check Option Bounds

In [46]:
ad = ArbitrageDetector(0.2)
bounds = ad.check_option_bounds(market_price, option_style, option.option_type, option.S, option.K, option.T, option.r, option.q)

print(js.dumps(bounds, indent=4))

{
    "market price": 44.675,
    "lower_bound": 44.60178613489407,
    "upper_bound": 264.57715956833425,
    "arbitrage exists": false,
    "profit": 0
}


### Set Up Box Spread

In [47]:
ticker = 'aapl'
stock = StockOptionsData(ticker)

K1 = 220
K2 = K1 + 5

call_option_K1, call_price_K1 = stock.get_relevant_options_data("call", K1)
call_option_K2, call_price_K2 = stock.get_relevant_options_data("call", K2)
print(f"Market Price {K1}: {call_price_K1}, Market Price {K2}: {call_price_K2}")

put_option_K1, put_price_K1 = stock.get_relevant_options_data("put", K1)
put_option_K2, put_price_K2 = stock.get_relevant_options_data("put", K2)
print(f"Market Price {K1}: {put_price_K1}, Market Price {K2}: {put_price_K2}")

rows = ["Call Price", "Put Price"]
cols = [f"Strike {K1}", f"Strike {K2}"]
df = pd.DataFrame(index=rows, columns=cols)

df.loc["Call Price"] = [call_price_K1, call_price_K2]
df.loc["Put Price"] = [put_price_K1, put_price_K2]

display(df)

Market Price 220: 44.675, Market Price 225: 39.599999999999994
Market Price 220: 0.005, Market Price 225: 0.04


,Strike 220,Strike 225
Call Price,44.675,39.6
Put Price,0.005,0.04


### Check For Box Spread Arbitrage

In [48]:
ad = ArbitrageDetector(0.2)
bs_arb = ad.box_spread(call_price_K1, call_price_K2, put_price_K1, put_price_K2, K1, K2, call_option_K1.T, stock.get_risk_free_rate())

print(js.dumps(bs_arb, indent=4))

{
    "arbitrage exists": false,
    "profit (pv)": -16.753440305305457
}
